In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
data = pd.read_excel('../data/data.xlsx')
data['Unnamed: 0'] = pd.to_datetime(data['Unnamed: 0'])
data = data.set_index('Unnamed: 0')
data.index.name = None
data.head()

In [ ]:
pre = data.groupby([data.index.year, data.index.month]).agg(["first", "last"])

data_pct_m = pd.DataFrame(index = data.groupby(data.index.to_period('M')).apply(lambda x: x.index.max()).values
                     , columns = data.columns)
for i in data.columns:
    data_pct_m[i] = ( (pre[i]['last'] - pre[i]['first']) / pre[i]['first'] ).values

print(data_pct_m.shape)
data_pct_m.head()

In [ ]:
data_pct = data.pct_change().dropna()

pdata = data[:366]
pdata_pct = pdata.pct_change().dropna()
pdata_pct_m = data_pct_m[:12]

In [ ]:
import random

In [ ]:
#data slicer
indmin = data.groupby(data.index.to_period('M')).apply(lambda x: x.index.min()).values
indmax = data.groupby(data.index.to_period('M')).apply(lambda x: x.index.max()).values

#itg lists with frames
periods = []
cors = []

p_periods = []

for i in range(0, 61):
    #data slicer
    period = data.loc[indmin[i]:indmax[i+11]]
    period_corr = period.corr()
    
    cors.append(period_corr.values.astype(np.float32))
    periods.append(period.values.astype(np.float32))
    
    #choose 10 assets:max corr, 10 assets:min corr, 5 assets:randomly
    abs_corr = period_corr.abs().sum().sort_values()

    asset_labels = []
    asset_labels += abs_corr[:10].index.to_list()
    asset_labels += abs_corr.index[random.sample(range(11, len(data.columns)-10), 5)].to_list()
    asset_labels += abs_corr[-10:].index.to_list()

    p_periods.append(period[asset_labels].values.astype(np.float32))

In [ ]:
import copulae
from copulae import GaussianCopula, StudentCopula

from scipy import stats

In [ ]:
def copula_eval(values):
    gc = GaussianCopula(dim=values.shape[1])
    gc.fit(values)
    gc_ll = gc.log_lik(values)
    gc_corr = gc.sigma
    return gc_corr
    #sc = StudentCopula(dim=values.shape[1])
    #sc.fit(values)
    #sc_ll = sc.log_lik(values)
    #sc_corr = sc.sigma
    
    #if gc_ll >= sc_ll:
    #    return gc_corr 
    #else:
    #    return sc_corr
    
def make_data_uniform(df):
    df = pd.DataFrame(df)
    us = []
    for col in df.columns:
        uniformer = stats.Uniform(a=min(df[col].values), b=max(df[col].values))
        us.append(uniformer.cdf(df[col]))
    return us

In [ ]:
from sklearn.decomposition import KernelPCA

In [ ]:
from tqdm import tqdm
from IPython.display import clear_output

In [ ]:
X_kpca_list = []
#X_kcpa_corr_list = []
X_kcpa_copula_cor_list = []
X_kcpa_copula_cor_large_list1 = []
X_kcpa_copula_cor_large_list2 = []

for i, (X, X_cor) in tqdm(enumerate(zip(periods, cors))):
    #encode
    kpca_sigmoid = KernelPCA(n_components=25, kernel="sigmoid", fit_inverse_transform=False)
    X_kcpa = kpca_sigmoid.fit_transform(make_data_uniform(X))
    X_kpca_list.append(X_kcpa)
    #copula eval
    X_kcpa_copula_corr = copula_eval(X_kcpa)
    X_kcpa_copula_cor_list.append(X_kcpa_copula_corr)
    #decode
    kpca_corr = KernelPCA(n_components=25, kernel="sigmoid", fit_inverse_transform=True)
    kpca_corr.fit(X_cor)
    
    X_kcpa_copula_large = kpca_corr.inverse_transform(X_kcpa_copula_corr)
    X_kcpa_copula_cor_large_list1.append(X_kcpa_copula_large)
    
    X_kcpa_copula_large = kpca_corr.inverse_transform(X_kcpa_copula_large.T)
    X_kcpa_copula_cor_large_list2.append(X_kcpa_copula_large)
    print(i)
clear_output(wait=True)

In [ ]:
for i in range(len(X_kpca_list)):
    np.savetxt('intermediate_data/kpca/X_kpca_list%s.gz' %i, X_kpca_list[i], delimiter=',')
    
for i in range(len(X_kcpa_copula_cor_list)):
    np.savetxt('intermediate_data/kpca/X_kcpa_copula_cor_list%s.gz' %i, X_kcpa_copula_cor_list[i], delimiter=',')
    
for i in range(len(X_kcpa_copula_cor_large_list1)):
    np.savetxt('intermediate_data/kpca/X_kcpa_copula_cor_large_list1%s.gz' %i, X_kcpa_copula_cor_large_list1[i], delimiter=',')
    
for i in range(len(X_kcpa_copula_cor_large_list2)):
    np.savetxt('intermediate_data/kpca/X_kcpa_copula_cor_large_list2%s.gz' %i, X_kcpa_copula_cor_large_list2[i], delimiter=',')